# NOTEBOOK COMPLETO DISTANCIA PROJETO FRAUDE

#### IMPORTANDO BIBLIOTECAS

In [51]:
import numpy as np
import pandas as pd

#calcular distância em km entre lat e long
import geopy.distance

#converter bytes to string
from io import StringIO 

#calculo dos coeficientes
from scipy.stats.stats import pearsonr
from sklearn.linear_model import LinearRegression
from scipy import stats

#conexão com o blob storage
from azure.storage.blob import BlockBlobService
#conexão com a table storage
from azure.data.tables import TableServiceClient

#### PEGAR ARQUIVO DO BLOB STORAGE

In [157]:
#ler o arquivo BaseDistancia2.csv do blob storage
accountName = "bluestoragefraude"
accountKey = "KlpYzFJrw/90kdD8n1M7Hj7/thvdAiOVOvMUswOds/Mv471eHQMO4bgobr4KL9gaifuXVU4pMnwMwmfUmvh4Vw=="
containerName = "fraude"
blobName = "BaseDistancia.csv"
blobService = BlockBlobService(account_name=accountName, account_key=accountKey)
result = blobService.get_blob_to_text(containerName, blobName)

#converter arquivo lido para dentro do df
idc = pd.read_csv(StringIO(result.content))

#### PRÉ-PROCESSAMENTO

In [160]:
#calcular a distância entre A e B de cada registro(linha)
lista = []
for c in range (len(idc)):
    lista.append(geopy.distance.distance(idc.iloc[c,8:10].tolist(), idc.iloc[c,10:].tolist()).km)
idc['distancia'] = lista

#### TREINO DE MODELO

In [161]:
#cria conexão com o table storage 
account_name = 'bluestoragefraude'
account_key = 'KlpYzFJrw/90kdD8n1M7Hj7/thvdAiOVOvMUswOds/Mv471eHQMO4bgobr4KL9gaifuXVU4pMnwMwmfUmvh4Vw=='
endpoint_suffix = 'core.windows.net'
account_connection_string = 'DefaultEndpointsProtocol=https;AccountName=bluestoragefraude;AccountKey=KlpYzFJrw/90kdD8n1M7Hj7/thvdAiOVOvMUswOds/Mv471eHQMO4bgobr4KL9gaifuXVU4pMnwMwmfUmvh4Vw==;EndpointSuffix=core.windows.net'

table_service = TableServiceClient.from_connection_string(conn_str=account_connection_string)
table_client = table_service.get_table_client(table_name="profilefraudedistancia")

In [194]:
def get_prediction_interval(prediction, y_test, test_predictions, pi=.95):
    #get standard deviation of y_test
    sum_errs = np.sum((y_test - test_predictions)**2)
    stdev = np.sqrt(1 / (len(y_test) - 2) * sum_errs)
    #get interval from standard deviation
    one_minus_pi = 1 - pi
    ppf_lookup = 1 - (one_minus_pi / 2)
    z_score = stats.norm.ppf(ppf_lookup)
    interval = z_score * stdev
    #generate prediction interval lower and upper bound cs_24
    lower, upper = prediction - interval, prediction + interval
    return upper

In [196]:
for i in idc['idPessoa']:
    
    customer = idc[idc['idPessoa'] == i]   
    y_data = idc['distancia'].loc[idc['idPessoa'] == i]
    
    x_data = np.arange(0, len(y_data), 1)
    x_data_composed = x_data.reshape(-1,1)

    #modelo
    lin_regression = LinearRegression().fit(x_data_composed, y_data)
    model_line = lin_regression.predict(x_data_composed)
    
    dias = x_data_composed
    nd = 1                                
    pc = dias[-1] + nd                      
    y_pred = lin_regression.intercept_ + lin_regression.coef_ * pc
    upper =  get_prediction_interval(y_pred, y_data, model_line)

    #coeficientes angular e linear
    b = lin_regression.intercept_ 
    a = float(lin_regression.coef_)
        
    #dicionario com dados de coeficientes do usuário
    customer = {u'PartitionKey': 'MF', 
                        u'RowKey': f'{i}', 
                        u'coef_angular': f'{a}',
                        u'coef_linear': f'{b}',
                        u'step': f'{len(x_data) + 1}',
                        u'upper': f'{float(upper)}'
                       }
    
    #delete registro com o id da row_key
    table_client.delete_entity(row_key=f'{i}', partition_key='MF')
    #adicionar registro com os dados do customer
    table_client.create_entity(entity=customer)

ZeroDivisionError: division by zero